In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
# Import EfficientNet
from tensorflow.keras.applications import EfficientNetB4
import matplotlib.pyplot as plt
import numpy as np
import os

base_data_dir = '/content/drive/MyDrive/tomatoes/leaf_classifier_dataset/'
train_dir = os.path.join(base_data_dir, 'train')

# تحديد أبعاد الصور وعدد العينات (batch size)
# Updated IMG_SIZE for EfficientNetB4
IMG_SIZE = (380, 380)
BATCH_SIZE = 128

# تحميل الداتا من المجلدات
# Keras will automatically name classes based on folder names (leaves, no_leaves)
# We split the data (80% for training, 20% for testing/validation)
train_dataset = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# عرض أسماء الفئات التي تم اكتشافها (يجب أن يطبع: ['leaves', 'no_leaves'])
class_names = train_dataset.class_names
print("Categories found:", class_names)

# تحسين أداء تحميل الداتا
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# -----------------------------------------------------------------
#  بناء المودل (استخدام Transfer Learning)
# -----------------------------------------------------------------

# تحميل مودل EfficientNetB4
# include_top=False يعني أننا لا نريد طبقة التصنيف الأخيرة منه
# Use the recommended input shape for EfficientNetB4
base_model = EfficientNetB4(input_shape=(380, 380, 3),
                         include_top=False,
                         weights='imagenet')

# "تجميد" طبقات المودل الأساسي حتى لا نعيد تدريبها
base_model.trainable = False

# بناء المودل الخاص بنا
inputs = Input(shape=(380, 380, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x) # طبقة لتصغير المخرجات
x = Dropout(0.2)(x) # طبقة لمنع الـ Overfitting
x = Dense(128, activation='relu')(x)

# The last layer:
# نستخدم 1 نيرون فقط (0 أو 1) و 'sigmoid' لأنه تصنيف ثنائي
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# -----------------------------------------------------------------
#  train model
# -----------------------------------------------------------------
print("\n--- Start train---")
# 10 epochs
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset
)
print("--- Finsh train---")

In [ ]:
# save model
model_save_path = '/content/drive/MyDrive/tomatoes/leaf_classifier_model.h5'
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")
# load model
loaded_model = load_model('/content/drive/MyDrive/tomatoes/leaf_classifier_model.h5')
print("Model loaded from lung_cancer_detector.keras")

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# 1. تحميل المودل الذي قمنا بحفظه
model_path = '/content/drive/MyDrive/tomatoes/leaf_classifier_model.h5'
model = tf.keras.models.load_model(model_path)

# 2. تحديد مسار الصورة التي تريد اختبارها
# !!! غيّر هذا المسار إلى صورة جديدة !!!
test_image_path = '/content/drive/MyDrive/tomatoes/leaf_classifier_dataset/train/no_leaves/180_IMG_06094608_jpg.rf.22fd94f6831a63029957ddef65d1a2b3.jpg'

# 3. تحميل الصورة وتجهيزها
img = image.load_img(test_image_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) # تحويلها لـ batch

prediction = model.predict(img_array)

# النتيجة ستكون رقم بين 0 و 1
score = prediction[0][0]

print(f"image: {test_image_path}")
print(f" prediction (Score): {score}")

# (Assume 'leaves' is class 0 and 'no_leaves' is class 1)
# إذا كان ['leaves', 'no_leaves']، إذن:
if score < 0.5:
    print("Result: Leaf ✅ (Class: leaves)")
else:
    print("Result: No leaf ❌ (Class: no_leaves)")
